In [1]:
from copy import deepcopy
import numpy as np

# Solver

Rubik's cube has the following index convention:

![Rubik indices](images/face-indices.png)

Given a state in numpy array of dimension 6 x 3 x 3, the index corresponds to the face indicated above. The central face in the image is the front face.
Each block is then a 3 x 3 array that corresponds to the cells.

Each cell colour is indicated by an integer between 0 to 5 (inclusive).

The correspondences between indices and faces are:
* 0: face up (U)
* 1: face left (L)
* 2: face front (F)
* 3: face right (R)
* 4: face down (D)
* 5: face back (B)

Each action is one of `"L"`, `"L'"`, `"R"`, `"R'"`, `"F"`, `"F'"`, `"B"`, `"B'"`, `"U"`, `"U'"`, `"D"`, `"D'"`, where action without apostrophe `'` is clockwise and action with apostrophe `'` is anticlockwise. Clockwise direction is determined by the rotation of the face.

In [26]:
class Rubik:
    def apply_action(state, action):
        """Applies the action on the state and returns a new state.
        Both `state` and returned state must be numpy array of 6 x 3 x 3 that represents a state of a Rubik's cube.
        Action must be one of "L", "L'", "R", "R'", "F", "F'", "B", "B'", "U", "U'", "D", "D'".

        First, deepcopy the initial state, and then call the respective functions that mutates the copied state.
        Then, return the copied state.
        """

        state = deepcopy(state)
        match action:
            case "L":
                return Rubik.rotate_left_clockwise(state)
            case "L'":
                return Rubik.rotate_left_anticlockwise(state)
            case "R":
                return Rubik.rotate_right_clockwise(state)
            case "R'":
                return Rubik.rotate_right_anticlockwise(state)
            case "F":
                return Rubik.rotate_front_clockwise(state)
            case "F'":
                return Rubik.rotate_front_anticlockwise(state)
            case "B":
                return Rubik.rotate_back_clockwise(state)
            case "B'":
                return Rubik.rotate_back_anticlockwise(state)
            case "U":
                return Rubik.rotate_up_clockwise(state)
            case "U'":
                return Rubik.rotate_up_anticlockwise(state)
            case "D":
                return Rubik.rotate_down_clockwise(state)
            case "D'":
                return Rubik.rotate_down_anticlockwise(state)
            case _:
                raise ValueError(f"Unrecognised action {action}")

    def rotate_face_clockwise(face):
        temp_corner = face[0][0]
        face[0][0] = face[2][0]
        face[2][0] = face[2][2]
        face[2][2] = face[0][2]
        face[0][2] = temp_corner

        temp_side = face[0][1]
        face[0][1] = face[1][0]
        face[1][0] = face[2][1]
        face[2][1] = face[1][2]
        face[1][2] = temp_side

    def rotate_face_anticlockwise(face):
        temp_corner = face[0][0]
        face[0][0] = face[0][2]
        face[0][2] = face[2][2]
        face[2][2] = face[2][0]
        face[2][0] = temp_corner

        temp_side = face[0][1]
        face[0][1] = face[1][2]
        face[1][2] = face[2][1]
        face[2][1] = face[1][0]
        face[1][0] = temp_side

    def rotate_left_clockwise(state):
        temp1, temp2, temp3 = state[0, :, 0]
        state[0, :, 0] = state[5, :, 0]
        state[5, :, 0] = state[4, :, 0]
        state[4, :, 0] = state[2, :, 0]
        state[2, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[1])
        return state

    def rotate_left_anticlockwise(state):
        temp1, temp2, temp3 = state[0, :, 0]
        state[0, :, 0] = state[2, :, 0]
        state[2, :, 0] = state[4, :, 0]
        state[4, :, 0] = state[5, :, 0]
        state[5, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[1])
        return state

    def rotate_right_clockwise(state):
        temp1, temp2, temp3 = state[0, :, 2]
        state[0, :, 2] = state[2, :, 2]
        state[2, :, 2] = state[4, :, 2]
        state[4, :, 2] = state[5, :, 2]
        state[5, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[3])
        return state

    def rotate_right_anticlockwise(state):
        temp1, temp2, temp3 = state[0, :, 2]
        state[0, :, 2] = state[5, :, 2]
        state[5, :, 2] = state[4, :, 2]
        state[4, :, 2] = state[2, :, 2]
        state[2, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[3])
        return state

    def rotate_front_clockwise(state):
        temp1, temp2, temp3 = state[0, 2, :]
        state[0, 2, :] = state[1, :, 2][::-1]
        state[1, :, 2][::-1] = state[4, 0, :][::-1]
        state[4, 0, :][::-1] = state[3, :, 0]
        state[3, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[2])
        return state

    def rotate_front_anticlockwise(state):
        temp1, temp2, temp3 = state[0, 2, :]
        state[0, 2, :] = state[3, :, 0]
        state[3, :, 0] = state[4, 0, :][::-1]
        state[4, 0, :][::-1] = state[1, :, 2][::-1]
        state[1, :, 2][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[2])
        return state

    def rotate_back_clockwise(state):
        temp1, temp2, temp3 = state[0, 0, :]
        state[0, 0, :] = state[3, :, 2]
        state[3, :, 2] = state[4, 2, :][::-1]
        state[4, 2, :][::-1] = state[1, :, 0][::-1]
        state[1, :, 0][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[5])
        return state

    def rotate_back_anticlockwise(state):
        temp1, temp2, temp3 = state[0, 0, :]
        state[0, 0, :] = state[1, :, 0][::-1]
        state[1, :, 0][::-1] = state[4, 2, :][::-1]
        state[4, 2, :][::-1] = state[3, :, 2]
        state[3, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[5])
        return state

    def rotate_up_clockwise(state):
        temp1, temp2, temp3 = state[1, 0, :]
        state[1, 0, :] = state[2, 0, :]
        state[2, 0, :] = state[3, 0, :]
        state[3, 0, :] = state[5, 2, :][::-1]
        state[5, 2, :][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[0])
        return state

    def rotate_up_anticlockwise(state):
        temp1, temp2, temp3 = state[1, 0, :]
        state[1, 0, :] = state[5, 2, :][::-1]
        state[5, 2, :][::-1] = state[3, 0, :]
        state[3, 0, :] = state[2, 0, :]
        state[2, 0, :] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[0])
        return state

    def rotate_down_clockwise(state):
        temp1, temp2, temp3 = state[1, 2, :]
        state[1, 2, :] = state[5, 0, :][::-1]
        state[5, 0, :][::-1] = state[3, 2, :]
        state[3, 2, :] = state[2, 2, :]
        state[2, 2, :] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[4])
        return state

    def rotate_down_anticlockwise(state):
        temp1, temp2, temp3 = state[1, 2, :]
        state[1, 2, :] = state[2, 2, :]
        state[2, 2, :] = state[3, 2, :]
        state[3, 2, :] = state[5, 0, :][::-1]
        state[5, 0, :][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[4])
        return state

    def turn_tuple(state):
        return tuple(Rubik.turn_face_tuple(state[i]) for i in range(6))

    def turn_face_tuple(face):
        return tuple(tuple(face[i]) for i in range(3))

    def is_terminal(state):
        centers = state[:, 1, 1]
        return np.all(state == centers[:, None][:, None])

In [3]:
test_face = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
])

test_face_rotated_clockwise = deepcopy(test_face)
Rubik.rotate_face_clockwise(test_face_rotated_clockwise)
test_face_rotated_clockwise_expected = np.array([
    [7, 4, 1],
    [8, 5, 2],
    [9, 6, 3],
])
assert np.all(test_face_rotated_clockwise == test_face_rotated_clockwise_expected)

test_face_rotated_anticlockwise = deepcopy(test_face)
Rubik.rotate_face_anticlockwise(test_face_rotated_anticlockwise)
test_face_rotated_anticlockwise_expected = np.array([
    [3, 6, 9],
    [2, 5, 8],
    [1, 4, 7],
])
assert np.all(test_face_rotated_anticlockwise == test_face_rotated_anticlockwise_expected)

In [3]:
test_cube = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],

    [[11, 12, 13],
     [14, 15, 16],
     [17, 18, 19]],

    [[21, 22, 23],
     [24, 25, 26],
     [27, 28, 29]],

    [[31, 32, 33],
     [34, 35, 36],
     [37, 38, 39]],

    [[41, 42, 43],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [57, 58, 59]],
])

In [5]:
test_left_clockwise_expected = np.array([
    [[51, 2, 3],
     [54, 5, 6],
     [57, 8, 9]],

    [[17, 14, 11],
     [18, 15, 12],
     [19, 16, 13]],

    [[1, 22, 23],
     [4, 25, 26],
     [7, 28, 29]],

    [[31, 32, 33],
     [34, 35, 36],
     [37, 38, 39]],

    [[21, 42, 43],
     [24, 45, 46],
     [27, 48, 49]],

    [[41, 52, 53],
     [44, 55, 56],
     [47, 58, 59]]
])

test_left_clockwise = Rubik.rotate_left_clockwise(deepcopy(test_cube))
assert np.all(test_left_clockwise_expected == test_left_clockwise)

In [7]:
test_left_anticlockwise_expected = np.array([
    [[21, 2, 3],
     [24, 5, 6],
     [27, 8, 9]],

    [[13, 16, 19],
     [12, 15, 18],
     [11, 14, 17]],

    [[41, 22, 23],
     [44, 25, 26],
     [47, 28, 29]],

    [[31, 32, 33],
     [34, 35, 36],
     [37, 38, 39]],

    [[51, 42, 43],
     [54, 45, 46],
     [57, 48, 49]],

    [[1, 52, 53],
     [4, 55, 56],
     [7, 58, 59]],
])

test_left_anticlockwise = Rubik.rotate_left_anticlockwise(deepcopy(test_cube))
assert np.all(test_left_anticlockwise_expected == test_left_anticlockwise)

In [4]:
test_right_clockwise_expected = np.array([
    [[1, 2, 23],
     [4, 5, 26],
     [7, 8, 29]],

    [[11, 12, 13],
     [14, 15, 16],
     [17, 18, 19]],

    [[21, 22, 43],
     [24, 25, 46],
     [27, 28, 49]],

    [[37, 34, 31],
     [38, 35, 32],
     [39, 36, 33]],

    [[41, 42, 53],
     [44, 45, 56],
     [47, 48, 59]],

    [[51, 52, 3],
     [54, 55, 6],
     [57, 58, 9]],
])

test_right_clockwise = Rubik.rotate_right_clockwise(deepcopy(test_cube))
assert np.all(test_right_clockwise_expected == test_right_clockwise)

In [5]:
test_right_anticlockwise_expected = np.array([
    [[1, 2, 53],
     [4, 5, 56],
     [7, 8, 59]],

    [[11, 12, 13],
     [14, 15, 16],
     [17, 18, 19]],

    [[21, 22, 3],
     [24, 25, 6],
     [27, 28, 9]],

    [[33, 36, 39],
     [32, 35, 38],
     [31, 34, 37]],

    [[41, 42, 23],
     [44, 45, 26],
     [47, 48, 29]],

    [[51, 52, 43],
     [54, 55, 46],
     [57, 58, 49]],
])

test_right_anticlockwise = Rubik.rotate_right_anticlockwise(deepcopy(test_cube))
assert np.all(test_right_anticlockwise_expected == test_right_anticlockwise)

In [6]:
test_front_clockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [19, 16, 13]],

    [[11, 12, 41],
     [14, 15, 42],
     [17, 18, 43]],

    [[27, 24, 21],
     [28, 25, 22],
     [29, 26, 23]],

    [[7, 32, 33],
     [8, 35, 36],
     [9, 38, 39]],

    [[37, 34, 31],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [57, 58, 59]],
])

test_front_clockwise = Rubik.rotate_front_clockwise(deepcopy(test_cube))
assert np.all(test_front_clockwise == test_front_clockwise_expected)

In [7]:
test_front_anticlockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [31, 34, 37]],

    [[11, 12, 9],
     [14, 15, 8],
     [17, 18, 7]],

    [[23, 26, 29],
     [22, 25, 28],
     [21, 24, 27]],

    [[43, 32, 33],
     [42, 35, 36],
     [41, 38, 39]],

    [[13, 16, 19],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [57, 58, 59]],
])

test_front_anticlockwise = Rubik.rotate_front_anticlockwise(deepcopy(test_cube))
assert np.all(test_front_anticlockwise == test_front_anticlockwise_expected)

In [8]:
test_back_clockwise_expected = np.array([
    [[33, 36, 39],
     [4, 5, 6],
     [7, 8, 9]],

    [[3, 12, 13],
     [2, 15, 16],
     [1, 18, 19]],

    [[21, 22, 23],
     [24, 25, 26],
     [27, 28, 29]],

    [[31, 32, 49],
     [34, 35, 48],
     [37, 38, 47]],

    [[41, 42, 43],
     [44, 45, 46],
     [11, 14, 17]],

    [[57, 54, 51],
     [58, 55, 52],
     [59, 56, 53]],
])

test_back_clockwise = Rubik.rotate_back_clockwise(deepcopy(test_cube))
assert np.all(test_back_clockwise == test_back_clockwise_expected)

In [15]:
test_back_anticlockwise_expected = np.array([
    [[17, 14, 11],
     [4, 5, 6],
     [7, 8, 9]],

    [[47, 12, 13],
     [48, 15, 16],
     [49, 18, 19]],

    [[21, 22, 23],
     [24, 25, 26],
     [27, 28, 29]],

    [[31, 32, 1],
     [34, 35, 2],
     [37, 38, 3]],

    [[41, 42, 43],
     [44, 45, 46],
     [39, 36, 33]],

    [[53, 56, 59],
     [52, 55, 58],
     [51, 54, 57]],
])

test_back_anticlockwise = Rubik.rotate_back_anticlockwise(deepcopy(test_cube))
assert np.all(test_back_anticlockwise == test_back_anticlockwise_expected)

In [10]:
test_up_clockwise_expected = np.array([
    [[7, 4, 1],
     [8, 5, 2],
     [9, 6, 3]],

    [[21, 22, 23],
     [14, 15, 16],
     [17, 18, 19]],

    [[31, 32, 33],
     [24, 25, 26],
     [27, 28, 29]],

    [[59, 58, 57],
     [34, 35, 36],
     [37, 38, 39]],

    [[41, 42, 43],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [13, 12, 11]],
])

test_up_clockwise = Rubik.rotate_up_clockwise(deepcopy(test_cube))
assert np.all(test_up_clockwise == test_up_clockwise_expected)

In [18]:
test_up_anticlockwise_expected = np.array([
    [[3, 6, 9],
     [2, 5, 8],
     [1, 4, 7]],

    [[59, 58, 57],
     [14, 15, 16],
     [17, 18, 19]],

    [[11, 12, 13],
     [24, 25, 26],
     [27, 28, 29]],

    [[21, 22, 23],
     [34, 35, 36],
     [37, 38, 39]],

    [[41, 42, 43],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [33, 32, 31]],
])

test_up_anticlockwise = Rubik.rotate_up_anticlockwise(deepcopy(test_cube))
assert np.all(test_up_anticlockwise == test_up_anticlockwise_expected)

In [12]:
test_down_clockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],

    [[11, 12, 13],
     [14, 15, 16],
     [53, 52, 51]],

    [[21, 22, 23],
     [24, 25, 26],
     [17, 18, 19]],

    [[31, 32, 33],
     [34, 35, 36],
     [27, 28, 29]],

    [[47, 44, 41],
     [48, 45, 42],
     [49, 46, 43]],

    [[39, 38, 37],
     [54, 55, 56],
     [57, 58, 59]],
])

test_down_clockwise = Rubik.rotate_down_clockwise(deepcopy(test_cube))
assert np.all(test_down_clockwise == test_down_clockwise_expected)

In [22]:
test_down_anticlockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],

    [[11, 12, 13],
     [14, 15, 16],
     [27, 28, 29]],

    [[21, 22, 23],
     [24, 25, 26],
     [37, 38, 39]],

    [[31, 32, 33],
     [34, 35, 36],
     [53, 52, 51]],

    [[43, 46, 49],
     [42, 45, 48],
     [41, 44, 47]],

    [[19, 18, 17],
     [54, 55, 56],
     [57, 58, 59]],
])

test_down_anticlockwise = Rubik.rotate_down_anticlockwise(deepcopy(test_cube))
assert np.all(test_down_anticlockwise == test_down_anticlockwise_expected)

In [24]:
test_tuple_expected = (
    ((1, 2, 3),
     (4, 5, 6),
     (7, 8, 9)),

    ((11, 12, 13),
     (14, 15, 16),
     (17, 18, 19)),

    ((21, 22, 23),
     (24, 25, 26),
     (27, 28, 29)),

    ((31, 32, 33),
     (34, 35, 36),
     (37, 38, 39)),

    ((41, 42, 43),
     (44, 45, 46),
     (47, 48, 49)),

    ((51, 52, 53),
     (54, 55, 56),
     (57, 58, 59)),
)

test_tuple = Rubik.turn_tuple(deepcopy(test_cube))
assert test_tuple == test_tuple_expected

In [28]:
assert not Rubik.is_terminal(deepcopy(test_cube))

test_cube_finished = np.array([
    [[1, 1, 1],
     [1, 1, 1],
     [1, 1, 1]],

    [[2, 2, 2],
     [2, 2, 2],
     [2, 2, 2]],

    [[4, 4, 4],
     [4, 4, 4],
     [4, 4, 4]],

    [[5, 5, 5],
     [5, 5, 5],
     [5, 5, 5]],

    [[3, 3, 3],
     [3, 3, 3],
     [3, 3, 3]],

    [[6, 6, 6],
     [6, 6, 6],
     [6, 6, 6]],
])

test_cube_unfinished = np.array([
    [[1, 1, 1],
     [1, 1, 1],
     [1, 1, 1]],

    [[2, 2, 2],
     [2, 2, 2],
     [2, 2, 2]],

    [[4, 4, 4],
     [4, 4, 4],
     [4, 4, 4]],

    [[5, 5, 5],
     [5, 5, 5],
     [5, 5, 5]],

    [[3, 3, 3],
     [3, 3, 3],
     [3, 3, 3]],

    [[6, 6, 6],
     [6, 6, 6],
     [6, 6, 1]],
])

assert Rubik.is_terminal(test_cube_finished)
assert not Rubik.is_terminal(test_cube_unfinished)